### Note: All the requested exercises are presented in this Notebook. The exercises are presented by sections and each one correspond to one of the tasks proposed.

# First Section

Table created using BeautifulSoup. All the preprocessing is done before turning the list into the dataframe.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).content
soup = BeautifulSoup(html)

In [3]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [4]:
df = pd.DataFrame(table_contents)

df['Borough'] = df['Borough'].replace( # code taken from the guidance file provided in the course
    {'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
     'EtobicokeNorthwest':'Etobicoke Northwest',
     'East YorkEast Toronto':'East York/East Toronto',
     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
 
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

# Second Section
### From geocoder
The process with the library __geocoder didn't work correctly__ for all the rows so __the csv document was used__ instead. However, the code to use the library is still available in the cell below.

In [6]:
import geocoder # import geocoder

def get_lat_lng_coords(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude

# df["Latitude"], df["Longitude"] = zip(*df["PostalCode"].map(get_lat_lng_coords))
# df.head()

### From file
From the file we merge based on the dataframe from _Section 1_. The result is shown below: Latitude and Longitud available.

In [7]:
df_coords = pd.read_csv("./Geospatial_Coordinates.csv") # read the file into a dataframe
df_coords.rename(columns={'Postal Code': 'PostalCode'}, inplace=True) # change column name to match df
df = df.merge(df_coords, left_on='PostalCode', right_on='PostalCode') # merge both dataframes over df
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Third Section

In [8]:
# filter data from Toronto
df = df[df['Borough'].apply(lambda x:'Toronto' in x)].reset_index(drop=True) 
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [9]:
import folium # map rendering library
from folium import plugins
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [10]:
address = 'Toronto' # code taken from the Notebooks presented in the course replacing NY by Toronto

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Map showing all the points in Toronto

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Map clustered. When zoom is applied the objects inside are shown. 
#### The library allow us do this automatically.

In [12]:
# let's start again with a clean copy of the map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# instantiate a mark cluster object
cluster = plugins.MarkerCluster().add_to(map_toronto)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    #label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location = [lat, lng],
        icon=None,
        popup=label,
    ).add_to(cluster) 

# display map
map_toronto